<a href="https://colab.research.google.com/github/mathtoledo/dsbd/blob/main/UFPR_DS2022_MatheusAllanToledo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando o DuckDB

In [ ]:
!pip install 'duckdb==0.6'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.4 MB 71 kB/s 


# Carregando os arquivos necessários

Dados metereológicos, dia a dia, da cidade de NY do ano de 2016

In [ ]:
!wget "https://pdet.github.io/assets/data/weather.csv"

--2022-12-01 20:57:48--  https://pdet.github.io/assets/data/weather.csv
Resolving pdet.github.io (pdet.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to pdet.github.io (pdet.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11145 (11K) [text/csv]
Saving to: ‘weather.csv’

weather.csv         100%[===================>]  10.88K  --.-KB/s    in 0s      

2022-12-01 20:57:48 (68.2 MB/s) - ‘weather.csv’ saved [11145/11145]



Dados das corridas de taxi do mês de Janeiro de 2016 na cidade de NY

In [ ]:
!wget "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-01.parquet"

--2022-12-01 20:57:51--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.51, 65.8.245.50, 65.8.245.178, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151251087 (144M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2016-01.parquet’

yellow_tripdata_201 100%[===================>] 144.24M   125MB/s    in 1.2s    

2022-12-01 20:57:52 (125 MB/s) - ‘yellow_tripdata_2016-01.parquet’ saved [151251087/151251087]



# Importar as Bibliotecas

In [ ]:
import pandas as pd
import duckdb
# acrescente aqui as outras bibliotecas eventualmente necessárias ao seu projeto


# Carregar os Dados nas tabelas

In [ ]:
# Conexão com a base
duck_conn = duckdb.connect('store.db')

# Existem várias maneiras de se ler os dados e criar tabelas relacionais a partir deles
# Neste exemplo vamos ler diretamente os dados do formato original e criar uma tabela.

# Criando tabelas a partir dos dados lidos
duck_conn.execute("CREATE TABLE cab_trip as SELECT * FROM 'yellow_tripdata_2016-01.parquet'")
duck_conn.execute("CREATE TABLE weather as SELECT * FROM 'weather.csv'")

# Obtendo e mostrando a descrição dos dados
duck_rel = duckdb.from_parquet('yellow_tripdata_2016-01.parquet')
print(duck_rel)

duck_rel = duckdb.from_csv_auto('weather.csv')
print(duck_rel)


---------------------
--- Relation Tree ---
---------------------
parquet_scan(yellow_tripdata_2016-01.parquet)

---------------------
-- Result Columns  --
---------------------
- VendorID (BIGINT)
- tpep_pickup_datetime (TIMESTAMP)
- tpep_dropoff_datetime (TIMESTAMP)
- passenger_count (BIGINT)
- trip_distance (DOUBLE)
- RatecodeID (BIGINT)
- store_and_fwd_flag (VARCHAR)
- PULocationID (BIGINT)
- DOLocationID (BIGINT)
- payment_type (BIGINT)
- fare_amount (DOUBLE)
- extra (DOUBLE)
- mta_tax (DOUBLE)
- tip_amount (DOUBLE)
- tolls_amount (DOUBLE)
- improvement_surcharge (DOUBLE)
- total_amount (DOUBLE)
- congestion_surcharge (INTEGER)
- airport_fee (INTEGER)

---------------------
-- Result Preview  --
---------------------
VendorID	tpep_pickup_datetime	tpep_dropoff_datetime	passenger_count	trip_distance	RatecodeID	store_and_fwd_flag	PULocationID	DOLocationID	payment_type	fare_amount	extra	mta_tax	tip_amount	tolls_amount	improvement_surcharge	total_amount	congestion_surcharge	airport_fe

# Execução das Queryes

In [ ]:
# Seta a execução das buscas para utilizarem duas threads
duck_conn.execute('PRAGMA threads=2')

# Escrever a instrução select e executar, salvando o resultado em um array
ComandoSql = """SELECT AVG(passenger_count),
    AVG(trip_distance),
    AVG(fare_amount),
    AVG(total_amount)
FROM cab_trip """

Resultado = duck_conn.execute(ComandoSql).fetchall()

print(Resultado)

# Tips and Tricks
# Na instrução abaixo a expresão ::DATE está forçando a coluna "tpep_pickup_datetime" a ser convertida para um tipo de Data
ComandoSql = """SELECT AVG(trip_distance)
FROM cab_trip inner join weather on (cab_trip.tpep_pickup_datetime::DATE = weather.date)
 where total_amount < 10 and maximum_temperature < 50 """

#Criando um sinônimo para as tabelas
duck_taxi = duck_conn.table('cab_trip')
duck_weather = duck_conn.table('weather')

print(duck_weather)

[(1.6709121548725927, 4.497235855589042, 12.487202307880976, 15.641974693181337)]
---------------------
--- Relation Tree ---
---------------------
Scan Table [cab_trip]

---------------------
-- Result Columns  --
---------------------
- VendorID (BIGINT)
- tpep_pickup_datetime (TIMESTAMP)
- tpep_dropoff_datetime (TIMESTAMP)
- passenger_count (BIGINT)
- trip_distance (DOUBLE)
- RatecodeID (BIGINT)
- store_and_fwd_flag (VARCHAR)
- PULocationID (BIGINT)
- DOLocationID (BIGINT)
- payment_type (BIGINT)
- fare_amount (DOUBLE)
- extra (DOUBLE)
- mta_tax (DOUBLE)
- tip_amount (DOUBLE)
- tolls_amount (DOUBLE)
- improvement_surcharge (DOUBLE)
- total_amount (DOUBLE)
- congestion_surcharge (INTEGER)
- airport_fee (INTEGER)

---------------------
-- Result Preview  --
---------------------
VendorID	tpep_pickup_datetime	tpep_dropoff_datetime	passenger_count	trip_distance	RatecodeID	store_and_fwd_flag	PULocationID	DOLocationID	payment_type	fare_amount	extra	mta_tax	tip_amount	tolls_amount	improvem

# Coloque a partir daqui as suas Respostas !!!

In [ ]:
# Questão 1
# Existem dados na tabela duck_weather que não seguem o padrão dos outros dados ou estão nulos, trate os dados e escreva uma breve explicação das decisões que você tomou

# Questão 2
# Existe corridas com mais de 4 passageiros na base, trate os dados e escreva uma breve explicação das decisões que você tomou

# Questão 3
# Crie e mostre um histograma com as temperaturas médias por mês do ano de 2016

# Questão 4
# Crie uma regressão mostrando a relação entre a distância percorrida (trip_distance) e o valor pago da tarifa (total_amount)

# Questão 5
# Crie e mostre uma relação entre a temperatura (average_temperature) e o tempo da viagem (tpep_pickup_datetime - tpep_dropoff_datetime)

# Questão 6
# Como prever a tarifa final sabendo a distância a ser percorrida e a hora da chamada do taxi ?


